---
- ToC
- Intro
- Methodik
- EDA + Process Model
    - Process durations (min max / avg) + metrics dom vs int 
    - -> bottleneck
    - Top 3 Traces to Process modell -> confromance check
    - rejected applications
    - unexpected behaviour

- missing details
- ML
- KPis
- process improvement
---